In [8]:
import pandas as pd
import digitalhub as dh

# Example of creating a DH project

In [9]:
project = dh.get_or_create_project(
    name="overtourism"
)

In [46]:
obj = project.get_dataitem("contamezzi")
obj

{'kind': 'table', 'metadata': {'project': 'overtourism', 'name': 'contamezzi', 'version': 'f6eafb73-20b1-4817-a49f-a2e0608b4369', 'created': '2024-11-12T09:33:50.44Z', 'updated': '2024-11-12T09:33:50.44Z', 'created_by': 'acelepija@fbk.eu', 'updated_by': 'acelepija@fbk.eu', 'embedded': False}, 'spec': {'path': 's3://datalake/projects/tourism/molveno/contamezzi.parquet'}, 'status': {'state': 'CREATED', 'files': [{'path': 's3://datalake/projects/tourism/molveno/contamezzi.parquet', 'name': 'contamezzi.parquet', 'size': 104064, 'content_type': 'application/octet-stream', 'last_modified': '2024-06-01T07:54:54.000+00:00'}]}, 'user': 'acelepija@fbk.eu', 'project': 'overtourism', 'name': 'contamezzi', 'id': 'f6eafb73-20b1-4817-a49f-a2e0608b4369', 'key': 'store://overtourism/dataitem/table/contamezzi:f6eafb73-20b1-4817-a49f-a2e0608b4369'}

In [14]:
obj.as_df()

,date,locType,locId,userCountry,userProfile,userCluster,value
0,2022-01-01,AL_2,22,ALB,COMMUTER,None,0
1,2022-01-01,AL_2,22,ALB,TOURIST,None,862
2,2022-01-01,AL_2,22,ALB,VISITOR,None,44
3,2022-01-01,AL_2,22,AND,TOURIST,None,0
4,2022-01-01,AL_2,22,AND,VISITOR,None,0
...,...,...,...,...,...,...,...
4584498,2024-03-31,TN_MKT_POI,STAZ_TREN,SRB,VISITOR,UE,0
4584499,2024-03-31,TN_MKT_POI,STAZ_TREN,SWE,TOURIST,UE,0
4584500,2024-03-31,TN_MKT_POI,STAZ_TREN,SWE,VISITOR,UE,0
4584501,2024-03-31,TN_MKT_POI,STAZ_TREN,USA,TOURIST,None,0


# Upload new datasets to the overtourism project

In [1]:
import boto3
from botocore.client import Config

In [28]:
ENDPOINT_URL = "http://minio:9000"
BUCKET_NAME = "datalake"

In [31]:
s3 = boto3.client('s3', endpoint_url=ENDPOINT_URL, config=Config(s3={'addressing_style': 'path'}))
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'{bucket["Name"]}')

Existing buckets:
datalake


In [45]:
response = s3.list_objects(
    Bucket='datalake',
    Prefix="projects/tourism"
)
for el in response["Contents"]:
    if not "_origin" in el["Key"]:
        first_index = el["Key"].rindex("/")
        point_index = el["Key"].index(".")
        name_di = el["Key"][first_index+1: point_index]
        project.new_dataitem(
                    name=name_di,
                    kind="table",
                    path=f"s3://datalake/{el['Key']}")

# Download overtourism datasets from the bucket

In [12]:
s3.download_file("datalake", "projects/tourism/vodafone/vodafone_attendences_ALL.parquet", "test.parquet")

In [13]:
test = pd.read_parquet("test.parquet")
test

,date,locType,locId,userCountry,userProfile,userCluster,value
0,2022-01-01,AL_2,22,ALB,COMMUTER,None,0
1,2022-01-01,AL_2,22,ALB,TOURIST,None,862
2,2022-01-01,AL_2,22,ALB,VISITOR,None,44
3,2022-01-01,AL_2,22,AND,TOURIST,None,0
4,2022-01-01,AL_2,22,AND,VISITOR,None,0
...,...,...,...,...,...,...,...
304378,2024-03-31,TN_MKT_POI,STAZ_ROVE,STR,TOURIST,STR,25
304379,2024-03-31,TN_MKT_POI,STAZ_ROVE,STR,VISITOR,STR,51
304380,2024-03-31,TN_MKT_POI,STAZ_TREN,STR,COMMUTER,STR,0
304381,2024-03-31,TN_MKT_POI,STAZ_TREN,STR,TOURIST,STR,50
